In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from scipy import interpolate
from importlib import reload
from sklearn import preprocessing
import multiprocessing as mp
from functools import partial
import scipy
import pydmd 

In [ ]:
DATA_NAME = 'WH06_SC14'
OUTPUT_DIR = f"{DATA_NAME}_DMD"
DATA_PATH = "/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/WH06/analysis_files/scene14/"

In [ ]:
"""Create output folder for figures """

os.mkdir(OUTPUT_DIR)

In [ ]:
"""Load the tracks"""


LOW_INTENSITY_THRESHOLD = 0.4

data = _tm.load_trackmate_dir(DATA_PATH)
df = _tm.merge_tracks_and_all(data)
df = _tm.clean_up_trackSpots(df, LOW_INTENSITY_THRESHOLD)
print(df.shape)

In [1]:

def build_tensor(df, subregion_div):

    frame = (int(df['POSITION_X'].max()), int(df['POSITION_Y'].max()))
    
    x_iter = np.linspace(0, frame[0], subregion_div[0]+1, dtype=int)
    y_iter = np.linspace(0, frame[1], subregion_div[1]+1, dtype=int)

    
    TIME = df['FRAME'].max()
    
    T = np.zeros((TIME, subregion_div[0], subregion_div[1]))
    
    for t in range(TIME):
    
        for i, x_coord in enumerate(x_iter[1:]):
            prev_x_coord = x_iter[i]

            for j, y_coord in enumerate(y_iter[1:]):
                prev_y_coord = y_iter[j]
                
                mask = (df['POSITION_X'] >= prev_x_coord) & (df['POSITION_X'] < x_coord) & \
                (df['POSITION_Y'] >= prev_y_coord) & (df['POSITION_Y'] < y_coord) & \
                (df['FRAME'] == t)
            
                hits = len(df[mask])
            
                T[t, j, i] = hits
                
    return list(T)


    
subregion_div = (50, 50)    
tensor = build_tensor(df, subregion_div)  
plt.imshow(tensor[5], origin='lower')

NameError: name 'df' is not defined

In [2]:

dmd = pydmd.DMD(svd_rank=3, tlsq_rank=3, opt=True)
dmd.fit(tensor)
dmd.plot_modes_2D(figsize=(12,5))

NameError: name 'pydmd' is not defined

In [3]:
for i, mode in enumerate(dmd.modes.T):
    plt.plot(range(subregion_div[0]*subregion_div[1]), mode.real)
    plt.title('Modes')
plt.show()

for dynamic in dmd.dynamics:
    plt.plot(range(154), dynamic.real)
    plt.title('Dynamics')
plt.show()

NameError: name 'dmd' is not defined

In [ ]:
compute_integral = scipy.integrate.trapz

dmd_states = [state.reshape(tensor[0].shape) for state in dmd.reconstructed_data.T]

original_int = [compute_integral(compute_integral(t)).real for t in tensor]
dmd_int = [compute_integral(compute_integral(state)).real for state in dmd_states]

matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (15, 5)

plt.plot(dmd.original_timesteps, original_int, 'bo', label='original snapshots')
plt.plot(dmd.dmd_timesteps, dmd_int, 'r.', label='dmd states')
plt.ylabel('Integral')
plt.xlabel('Time')
plt.grid()
leg = plt.legend()